In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import sklearn
import pytorch_lightning as pl
import warnings
from pytorch_lightning.loggers import MLFlowLogger
from pytorch_lightning.loggers.base import LightningLoggerBase
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from torch import nn
from torch.utils.data import DataLoader
from typing import Optional, List, Dict, Any, Callable

warnings.filterwarnings("ignore")
pl.utilities.seed.seed_everything(42)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
def normalize(data):
    scaler = MinMaxScaler()
    return scaler.fit_transform(data)

def standarize(data):
    scaler = StandardScaler()
    return scaler.fit_transform(data)

# Load Data

In [ ]:
class PimaIndiansDataModule(pl.LightningDataModule):
    def __init__(self, data_dir: str = "data/diabetes.csv", train_batch_size: int = 64, val_batch_size: int = 64, transforms: List[Callable] = [], no_batch: bool = False):
        super().__init__()
        self.data_dir = data_dir
        self.train_batch_size = train_batch_size
        self.val_batch_size = val_batch_size
        self.no_batch = no_batch
        self.transforms = transforms
        
    def prepare_data(self):
        self.full_data = pd.read_csv('data/diabetes.csv')
        #self.full_data = self.full_data.drop(['Age'], axis=1)
        X = self.full_data.values[:, :-1]
        y = self.full_data.values[:, -1]
        for transform in self.transforms:
            X = transform(X)
        self.transformed_data = X
            
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)
        X_train = torch.tensor(X_train).float()
        X_test = torch.tensor(X_test).float()
        y_train = torch.tensor(y_train).long()
        y_test = torch.tensor(y_test).long()
        
        self.train_data = (X_train, y_train)
        self.test_data = (X_test, y_test)
    
    def setup(self, stage: Optional[str] = None):
        if stage == 'fit' or stage is None:
            X_train, y_train = self.train_data
            X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, stratify=y_train)
            self.train_data = []
            self.val_data = []
            for x, y in zip(X_train, y_train):
                self.train_data.append((x, y))
                
            for x, y in zip(X_val, y_val):
                self.val_data.append((x, y))
            
            if self.no_batch:
                self.train_batch_size = len(self.train_data)
                self.val_batch_size = len(self.val_data)
        
        if stage == 'test' or stage is None:
            X_test, y_test = self.test_data
            self.test_data = []
            for x, y in zip(X_test, y_test):
                self.test_data.append((x, y))

    def train_dataloader(self):
        return DataLoader(self.train_data, self.train_batch_size, shuffle=True, num_workers=1, pin_memory=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_data, self.val_batch_size, shuffle=False, num_workers=1, pin_memory=True)
    
    def test_dataloader(self):
        return DataLoader(self.test_data, shuffle=False, num_workers=1, pin_memory=True)

In [ ]:
datamodule = PimaIndiansDataModule(no_batch=True, transforms=[normalize])
datamodule.prepare_data()

In [ ]:
pca = PCA()
values = pca.fit_transform(datamodule.transformed_data)

In [ ]:
heights = pca.explained_variance_ratio_
gen = iter(range(0, 9, 1))
labels = [next(gen) for _ in heights]
plt.bar(labels, heights)
len(heights)

In [ ]:
datamodule.full_data.head(5)

In [ ]:
datamodule.full_data.info()

In [ ]:
datamodule.full_data.describe()

In [ ]:
one_class = sum(datamodule.full_data.values[:, -1])
zero_class = datamodule.full_data['Outcome'].size - one_class
print(f'Class 0: {zero_class}')
print(f'Class 1: {one_class}')

# Visualisation

In [ ]:
def plot_hist(data, x, y, figsize=(13, 13), bins=15):
    fig, ax = plt.subplots(x, y, figsize=figsize)
    for i in range(data.transformed_data.shape[1]):
        ax[int(i / y), i % y].hist(data.transformed_data[:, i], bins=bins)
        ax[int(i / y), i % y].set_title(data.full_data.columns[i])
    plt.show()

In [ ]:
plot_hist(datamodule, 2, 4)

# Models

In [ ]:
class Classifier(pl.LightningModule):
    def __init__(self, model, lr=1e-4):
        super().__init__()
        self.model = model
        self.lr = lr
        self.test_acc = 0
        self.test_f1 = 0
    
    def forward(self, x):
        return self.model.forward(x)
        
    def shared_step(self, batch):
        x, y = batch
        pred = self(x)
        criterion = nn.CrossEntropyLoss()
        loss = criterion(pred, y)
        return pred, loss
        
    def training_step(self, train_batch, batch_idx):
        pred, loss = self.shared_step(train_batch)
        _, predicted = torch.max(pred.data, 1)
        _, y = train_batch
        return {'loss': loss, 'train_score': (predicted, y)}
    
    def training_epoch_end(self, outputs):
        loss = [output['loss'] for output in outputs]
        loss_nd = np.double(loss[-1].detach().cpu().numpy())
        
        results = [x['train_score'] for x in outputs]
        preds = []
        y = []
        for predicted, y_data in results:
            predicted = predicted.detach().cpu().numpy()
            y_data = y_data.detach().cpu().numpy()
            preds.extend(predicted)
            y.extend(y_data)
        f1 = f1_score(y, preds)
        self.log('loss', loss_nd, logger=True)
        self.log('train_f1', f1, prog_bar=True, logger=True)
    
    def validation_step(self, val_batch, val_batch_idx):
        pred, loss = self.shared_step(val_batch)
        _, predicted = torch.max(pred.data, 1)
        _, y = val_batch
        return {'val_loss': loss, 'val_score': (predicted, y)}
    
    def validation_epoch_end(self, outputs):
        results = [x['val_score'] for x in outputs]
        preds = []
        y = []
        for predicted, y_data in results:
            predicted = predicted.detach().cpu().numpy()
            y_data = y_data.detach().cpu().numpy()
            preds.extend(predicted)
            y.extend(y_data)
        f1 = f1_score(y, preds)
        self.log('val_f1', f1, prog_bar=True, logger=True)
        
    def test_step(self, test_batch, test_batch_idx):
        pred, loss = self.shared_step(test_batch)
        _, predicted = torch.max(pred.data, 1)
        _, y = test_batch
        return {'test_loss': loss, 'test_score': (predicted, y)}
    
    def test_epoch_end(self, outputs):
        results = [x['test_score'] for x in outputs]
        preds = []
        y = []
        for predicted, y_data in results:
            predicted = predicted.detach().cpu().numpy()
            y_data = y_data.detach().cpu().numpy()
            preds.extend(predicted)
            y.extend(y_data)
        acc = accuracy_score(y, preds)
        f1 = f1_score(y, preds)
        self.test_acc = acc
        self.test_f1 = f1
        self.log('accuracy', acc, prog_bar=True, logger=True)
        self.log('f1', f1, prog_bar=True, logger=True)
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)
        scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
        return [optimizer], [scheduler]

In [ ]:
class SpreadNNNoPool(pl.LightningModule):
    def __init__(self, in_size, out_size):
        super().__init__()
        
        self._nn = nn.Sequential(
            nn.Linear(in_size, in_size * 2),
            nn.BatchNorm1d(in_size * 2),
            nn.LeakyReLU(),
            
            nn.Linear(in_size * 2, in_size * 4),
            nn.BatchNorm1d(in_size * 4),
            nn.LeakyReLU(),
            
            nn.Linear(in_size * 4, in_size * 2),
            nn.BatchNorm1d(in_size * 2),
            nn.LeakyReLU(),
            
            nn.Linear(in_size * 2, int(in_size / 2)),
            nn.BatchNorm1d(int(in_size / 2)),
            nn.LeakyReLU(),
            
            nn.Linear(int(in_size / 2), out_size)
        )
        
        self.name = 'spreadnn_no_pool.ckpt'
    
    def forward(self, x):
        return self._nn(x)

In [ ]:
class SpreadNNNoPoolBig(pl.LightningModule):
    def __init__(self, in_size, out_size):
        super().__init__()
        
        in_size_2 = in_size * 2
        in_size_4 = in_size * 4
        in_size_8 = in_size * 8
        in_size_16 = in_size * 16
        self._nn = nn.Sequential(
            nn.Linear(in_size, in_size_4),
            nn.BatchNorm1d(in_size_4),
            nn.LeakyReLU(),
            
            nn.Linear(in_size_4, in_size_16),
            nn.BatchNorm1d(in_size_16),
            nn.LeakyReLU(),
            
            nn.Linear(in_size_16, in_size_8),
            nn.BatchNorm1d(in_size_8),
            nn.LeakyReLU(),
            
            nn.Linear(in_size_8, in_size_4),
            nn.BatchNorm1d(in_size_4),
            nn.LeakyReLU(),
            
            nn.Linear(in_size_4, in_size),
            nn.BatchNorm1d(in_size),
            nn.LeakyReLU(),
            
            nn.Linear(in_size, out_size)
        )
        
        self.name = 'spreadnn_no_pool_big.ckpt'
    
    def forward(self, x):
        return self._nn(x)

In [ ]:
class SpreadNNNoPoolGigaBig(pl.LightningModule):
    def __init__(self):
        super().__init__()
        
        self._nn = nn.Sequential(
            nn.Linear(8, 32),
            nn.BatchNorm1d(32),
            nn.LeakyReLU(),
            
            nn.Linear(32, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
            
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(),
            
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
            
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(),
            
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.LeakyReLU(),
            
            nn.Linear(32, 8),
            nn.BatchNorm1d(8),
            nn.LeakyReLU(),
            
            nn.Linear(8, 2)
        )
        
        self.name = 'spreadnn_no_pool_giga_big.ckpt'
    
    def forward(self, x):
        return self._nn(x)

In [ ]:
class SpreadNNNoPoolDoubleBig(pl.LightningModule):
    def __init__(self, in_size, out_size):
        super().__init__()
        
        in_size_2 = in_size * 2
        in_size_4 = in_size * 4
        in_size_8 = in_size * 8
        in_size_16 = in_size * 16
        self._nn = nn.Sequential(
            nn.Linear(in_size, in_size_4),
            nn.BatchNorm1d(in_size_4),
            nn.LeakyReLU(),
            
            nn.Linear(in_size_4, in_size_16),
            nn.BatchNorm1d(in_size_16),
            nn.LeakyReLU(),
            
            nn.Linear(in_size_16, in_size_16),
            nn.BatchNorm1d(in_size_16),
            nn.LeakyReLU(),
            
            nn.Linear(in_size_16, in_size_8),
            nn.BatchNorm1d(in_size_8),
            nn.LeakyReLU(),
            
            nn.Linear(in_size_8, in_size_4),
            nn.BatchNorm1d(in_size_4),
            nn.LeakyReLU(),
            
            nn.Linear(in_size_4, in_size),
            nn.BatchNorm1d(in_size),
            nn.LeakyReLU(),
            
            nn.Linear(in_size, out_size)
        )
        
        self.name = 'spreadnn_no_pool_double_big.ckpt'
    
    def forward(self, x):
        return self._nn(x)

In [ ]:
class SpreadNNSmall(pl.LightningModule):
    def __init__(self):
        super().__init__()
        
        self._nn = nn.Sequential(
            nn.Linear(8, 16),
            nn.BatchNorm1d(16),
            nn.LeakyReLU(),
            
            nn.Linear(16, 8),
            nn.BatchNorm1d(8),
            nn.LeakyReLU(),
            
            nn.Linear(8, 4),
            nn.BatchNorm1d(4),
            nn.LeakyReLU(),
            
            nn.Linear(4, 2)
        )
        
        self.name = 'spreadnn_small.ckpt'
    
    def forward(self, x):
        return self._nn(x)

# Evaluation

In [ ]:
def generate(elem, count, step):
    current = elem
    output = []
    for _ in range(count):
        output.append(current)
        current += step
    return output

In [ ]:
class Evaluator:
    def __init__(self, models: List[pl.LightningModule], version: str, trainer_params: Dict[str, Any] = {}, logger: LightningLoggerBase = MLFlowLogger, logger_params: Dict[str, Any] = {},
                 callbacks=[], callback_params={}, train_dataset = None, test_dataset = None):
        self.models = models
        self.classifiers = [Classifier(model) for model in models]
        self.train_dataset = train_dataset
        self.test_dataset = test_dataset
        self.version = version
        self.paths = ['model/k_fold/' + version + model.name for model in models]
        self.loggers = [logger(**logger_params) for _ in models]
        self.callbacks = []
        for _ in models:
            self.callbacks.append([callback(**callback_params) for callback in callbacks])
        self.trainer_params = trainer_params
        self.acc_scores = {}
        self.f1_scores = {}
        for model in models:
            self.acc_scores[model.name] = []
            self.f1_scores[model.name] = []
    
    def train(self, min_lr: float, datamodule: pl.LightningDataModule, transforms: List[Callable], find_lr: bool = True, verbose: bool = False):
        for path, classifier, logger, callback in zip(self.paths, self.classifiers, self.loggers, self.callbacks):
            self.trainer = pl.Trainer(logger=logger, callbacks=callback, **self.trainer_params)
            if find_lr:
                lr_finder = self.trainer.tuner.lr_find(classifier, min_lr=min_lr, datamodule=datamodule(train_dataset, test_dataset), early_stop_threshold=None)
                classifier.lr = lr_finder.suggestion()
            else:
                classifier.lr = min_lr

            if verbose and find_lr:
                print(f'Best lr: {classifier.lr}')

            self.trainer.fit(classifier, datamodule(train_dataset, test_dataset))
            self.trainer.save_checkpoint(path)
    
    def test(self, datamodule: pl.LightningDataModule, transforms: List[Callable]):
        for path, model, logger in zip(self.paths, self.models, self.loggers):
            self.trainer = pl.Trainer(logger=logger, **self.trainer_params)
            classifier = Classifier.load_from_checkpoint(path, model=model)
            self.trainer.test(classifier, datamodule=datamodule(train_dataset, test_dataset))
            self.acc_scores[classifier.model.name].append(classifier.test_acc)
            self.f1_scores[classifier.model.name].append(classifier.test_f1)

In [ ]:
class LinearSearch:
    def __init__(self, grid_params, combinations):
        self.grid_params = grid_params
        self.combinations = combinations
        self.acc_scores = []
        self.f1_scores = []
    
    def search(self, models, in_size, out_size, datamodule, versions, min_lr, transforms, find_lr, verbose, callbacks=[], callback_params={}, train_dataset = None, test_dataset = None):
        for i in range(self.combinations):
            obj_models = [model(in_size, out_size) for model in models]
            params = {}
            for key in self.grid_params.keys():
                params[key] = self.grid_params[key][i]
            print(params)
            evaluator = Evaluator(obj_models, versions[i], params, callbacks=callbacks, callback_params=callback_params, train_dataset=train_dataset, test_dataset=test_dataset)
            evaluator.train(min_lr, datamodule, transforms, find_lr=find_lr, verbose=verbose)
            evaluator.test(datamodule, transforms)
            self.acc_scores.append(evaluator.acc_scores)
            self.f1_scores.append(evaluator.f1_scores)

In [ ]:
models = [SpreadNNNoPoolBig, SpreadNNNoPoolDoubleBig]
min_lr = 1e-03
in_size = 8
out_size = 2
datamodule = PimaIndiansDataModule
transforms = [normalize]
early_stopping_params = {'monitor': 'loss', 'min_delta': 0.001, 'patience': 6}
callbacks = [EarlyStopping]
params_count = 3
params = {
    'gpus': generate(1, params_count, 0),
    'max_epochs': [30, 75, 100],
    'gradient_clip_val': generate(0.5, params_count, 0.),
    'stochastic_weight_avg': generate(True, params_count, False),
    'amp_level': generate('O3', params_count, ''),
    'precision': generate(16, params_count, 0)
}
versions = [str(max_epoch) + 'epoch_' for max_epoch in params['max_epochs']]
print(versions)

In [ ]:
for i in range(10):
    linear_search = LinearSearch(params, params_count)
    loop_versions = [str(i) + '_' + version for version in versions]
    linear_search.search(models, in_size, out_size, datamodule, loop_versions, min_lr, transforms, False, False, callbacks, early_stopping_params)

In [ ]:
class DataModule(pl.LightningDataModule):
    def __init__(self, train_dataset, test_dataset):
        super().__init__()
        self.train_dataset = train_dataset
        self.test_dataset = test_dataset
        
    def prepare_data(self):
        train = []
        X_train = torch.tensor(self.train_dataset.values[:, :-1]).float()
        y_train = torch.tensor(self.train_dataset.values[:, -1]).long()
        for x, y in zip(X_train, y_train):
            train.append((x, y))
        self.train_dataset = train
        
        test = []
        X_test = torch.tensor(self.test_dataset.values[:, :-1]).float()
        y_test = torch.tensor(self.test_dataset.values[:, -1]).long()
        for x, y in zip(X_test, y_test):
            test.append((x, y))
        self.test_dataset = test
    
    def setup(self, stage: Optional[str] = None):
        pass

    def train_dataloader(self):
        return DataLoader(self.train_dataset, len(self.train_dataset), shuffle=True, num_workers=1, pin_memory=True)
    
    def test_dataloader(self):
        return DataLoader(self.test_dataset, len(self.test_dataset), shuffle=True, num_workers=1, pin_memory=True)

In [ ]:
skf = StratifiedKFold(n_splits=10)
datamodule = PimaIndiansDataModule(no_batch=True, transforms=[normalize])
datamodule.prepare_data()
X = datamodule.transformed_data
y = datamodule.full_data.values[:, -1]
acc_scores = []
f1_scores = []
for _ in range(params_count):
    acc_scores.append(list())
    f1_scores.append(list())
    
for i, idx in enumerate(skf.split(X, y)):
    train_idx, test_idx = idx
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    train_dataset = pd.DataFrame(X_train)
    train_dataset['Output'] = y_train
    test_dataset = pd.DataFrame(X_test)
    test_dataset['Output'] = y_test
    datamodule = DataModule
    linear_search = LinearSearch(params, params_count)
    loop_versions = [str(i) + '_' + version for version in versions]
    linear_search.search(models, in_size, out_size, datamodule, loop_versions, min_lr, transforms, False, False, callbacks, early_stopping_params, train_dataset=train_dataset, test_dataset=test_dataset)
    for j, metrics in enumerate(zip(linear_search.acc_scores, linear_search.f1_scores)):
        acc_scores[j].append(metrics[0]) # 9 list po 10 słowników po 2 modele
        f1_scores[j].append(metrics[1])

In [ ]:
models = acc_scores[0][0].keys()
final_models_acc_scores = []
final_models_f1_scores = []
for acc, f1 in zip(acc_scores, f1_scores):
    for model in models:
        model_acc_scores = [models_dict[model] for models_dict in acc]
        model_f1_scores = [models_dict[model] for models_dict in f1]
        final_models_acc_scores.append(np.mean(model_acc_scores))
        final_models_f1_scores.append(np.mean(model_f1_scores))

In [ ]:
for acc, f1 in zip(final_models_acc_scores, final_models_f1_scores):
    result = {'accuracy': acc, 'f1': f1}
    print(result)